In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,CuDNNLSTM
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_train = []
scaled_test = []
for i in range(train_X.shape[0]):
  scaled_train.append(scaler.fit_transform(train_X[i]))
for i in range(test_X.shape[0]):  
  scaled_test.append(scaler.fit_transform(test_X[i]))

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [4]:
X_train = np.array(scaled_train)
X_test = np.array(scaled_test)
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [6]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=X_train.shape[1:],return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(128))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

In [7]:
optimiser = tf.keras.optimizers.Adam(lr=0.001, decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=optimiser,
             metrics=['accuracy'])

In [9]:
import time
start = time.time()
model.fit(X_train,
          train_y,
          epochs=3,
          validation_data=(X_test, test_y), verbose=1)
end = time.time()
print('Net time using P100 GPU : ', end-start, ' secs.')

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 13s 221us/step - loss: 0.0647 - acc: 0.9827 - val_loss: 0.0528 - val_acc: 0.9845
Epoch 2/3
60000/60000 [==============================] - 13s 221us/step - loss: 0.0522 - acc: 0.9858 - val_loss: 0.0550 - val_acc: 0.9846
Epoch 3/3
60000/60000 [==============================] - 13s 221us/step - loss: 0.0434 - acc: 0.9878 - val_loss: 0.0577 - val_acc: 0.9850
Net time using P100 GPU :  39.82401418685913  secs.
